In [ ]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from mld_dynamics import mld, h, x_min, x_max, fc_min, fc_max, d, l
from warm_start_hmpc.controller import HybridModelPredictiveController
from warm_start_hmpc.mcais import solve_dare, mcais
from visualizer import vis, animate

In [ ]:
# controller horizon
T = 20

# weight matrices
C = np.eye(mld.nx) * h
# C = np.diag([1.,1.,1.,1.]) * h
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T * h

# LQR cost to go
Bu = mld.B[:,0:1]
Du = D[:,0:1]
P, K = solve_dare(mld.A, Bu, C.dot(C), Du.dot(Du))
C_T = np.linalg.cholesky(P).T
objective = [C, D, C_T]

# teminal set
A_cl = mld.A + Bu.dot(K)
lhs = mld.F + mld.G[:,0:1].dot(K)
rhs = mld.h
terminal_set = mcais(A_cl, lhs, rhs, verbose=True)

In [ ]:
# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])
# x0 = np.array([0., 0., 0., 0.5])

## Solve in closed loop

In [ ]:
# set up simulation
x = [x0]
u = []
warm_start = None

T_sim = 50
for t in range(T_sim):
    print('Time step %d.'%t, end='\r')
    
    # solve miqp
    solution, leaves, n_nodes = controller.feedforward(
        x[-1],
        warm_start=warm_start,
        printing_period=None
    )
    
    # reorganize solution
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    u0 = np.concatenate((uc0, ub0))
    
    # generate warm start
    warm_start = controller.construct_warm_start(
        leaves,
        x[-1],
        uc0,
        ub0,
        np.zeros(4)
    )
    
    # retrieve closed-loop trajectory
    x.append(solution.variables['x'][1])
    u.append(uc0[0])

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 12})
plt.rc('text.latex', preamble=r'\usepackage{stackengine}')

fig, axes = plt.subplots(2, sharex=True, figsize=(6,3.5))
t = range(T_sim+1)
# axes[0].step(t, [fc_min[0]]*(T_sim+1), label=r'Force bound \b{$u$}$_1$', color='#d62728', linestyle='--')
# axes[0].step(t, [fc_min[0]]*(T_sim+1), label=r'Force bound $\underline u_1$', color='#d62728', linestyle='--')
axes[0].step(t, [fc_min[0]]*(T_sim+1), label=r'Force bound \stackunder[1.2pt]{$u$}{\rule{.8ex}{.075ex}}$_1$', color='#d62728', linestyle='--')
axes[0].step(t, [u[0]] + u, label=r'Force on cart $u_1$', color='#1f77b4')
handles, labels = axes[0].get_legend_handles_labels()
order = [1,0]
axes[0].legend([handles[i] for i in order],[labels[i] for i in order],loc='right')
# axes[0].set_ylabel(r'$u_1$')
axes[0].set_xlim((0, T_sim))
yticks = np.linspace(-1, .4, 8)
axes[0].set_yticks(yticks)

C = np.array([[1., -l, 0., 0.]])
y = [C.dot(xt) for xt in x]
axes[1].step(t, [d]*(T_sim+1), label=r'Right wall $d$', color='#2ca02c', linestyle='--')
axes[1].plot(t, y, label=r'Pole tip $x_1-lx_2$', color='#1f77b4')
# axes[1].set_ylabel(r'$x_{1}-lx_{2}$')
axes[1].set_xlabel(r'Time step $\tau$')
handles, labels = axes[1].get_legend_handles_labels()
order = [1,0]
axes[1].legend([handles[i] for i in order],[labels[i] for i in order],loc='right')
yticks = np.linspace(0, .6, 7)
axes[1].set_yticks(yticks)

xticks = range(0, 51, 5)
plt.xticks(xticks)
axes[0].grid(True, color=np.ones(3)*.85)
axes[1].grid(True, color=np.ones(3)*.85)

fig.tight_layout()
plt.savefig('simulation.pdf', bbox_inches='tight')

## Statistical Analysis

In [ ]:
e_sd = 0.03# 0.001, 0.003, 0.01
N_sim = 50
N_samples = 100
i_0 = 0

if i_0 == 0:
    nodes = []
    nodes_ws = []
    nodes_gurobi = []
    len_ws = []
    errors = []
else:
    nodes = list(np.load('data/nodes_sd_%.3f.npy'%e_sd))
    nodes_ws = list(np.load('data/nodes_ws_sd_%.3f.npy'%e_sd))
#     nodes_gurobi = list(np.load('data/nodes_gurobi_sd_%.3f.npy'%e_sd))
    len_ws = list(np.load('data/len_ws_sd_%.3f.npy'%e_sd))
    errors = list( np.load('data/errors_sd_%.3f.npy'%e_sd))

with open('data/solve_log_sd_%.3f.log'%e_sd, 'w') as f:

    f.write('Error standard deviation %.3f\n\n'%e_sd) 
    f.flush()

    i = i_0

    while len(nodes) < N_samples:
        np.random.seed(i)

        f.write('\nRollout %d\n'%len(nodes))
        f.write('Iteration n.%d\n\n'%i)
        f.flush()
        rollout_success = True

        nodes_i = []
        nodes_ws_i = []
        nodes_gurobi_i = []
        len_ws_i = []
        errors_i = []

        x_sim = [x0]
        ws = None

        for t in range(N_sim):
            print((i,t), end='\r')
            f.write('Time step %d '%t)
            f.flush()

#             # project previous state in feasible set
#             x_prev = np.minimum(np.maximum(x_sim[-1], x_min), x_max)
            
#             # solve with gurobi
#             try:
#                 x_gurobi, cost_gurobi, n_nodes_gurobi = controller.feedforward_gurobi(
#                     x_sim[-1],
#                     {'OutputFlag': 0}
#                 )
#             except:
#                 rollout_success = False
#                 f.write('\nUnseccessful rollout, gurobi broke.')
#                 break
#             nodes_gurobi_i.append(n_nodes_gurobi)
#             f.write(str(int(n_nodes_gurobi)) + ' ')
#             f.flush()
                
            # solve without warm start
            try:
                solution, leaves, n_nodes = controller.feedforward(
                    x_sim[-1],
                    printing_period=None
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, cold-started BB broke.')
                break
            nodes_i.append(n_nodes)
            f.write(str(n_nodes) + ' ')
            f.flush()

            # solve with warm start
            try:
                solution_ws, leaves_ws, n_nodes_ws = controller.feedforward(
                    x_sim[-1],
                    warm_start=ws,
                    printing_period=None
                )
            except:
                rollout_success = False
                f.write('\nUnseccessful rollout, cold-started BB broke.')
                break
            nodes_ws_i.append(n_nodes_ws)
            f.write(str(n_nodes_ws) + ' ')
            f.flush()

            # check feasibility
            if solution is None:
                break
            assert np.isclose(solution.objective, solution_ws.objective)
#             assert np.isclose(solution.objective, cost_gurobi)

            # generate random error of specified norm
            e_t = e_sd * np.multiply(np.random.randn(mld.nx), x_max)
            errors_i.append(e_t)

            # generate warm start
            ws = controller.construct_warm_start(
                leaves_ws,
                solution_ws.variables['x'][0],
                solution.variables['uc'][0],
                solution.variables['ub'][0],
                e_t
            )
            len_ws_i.append(len(ws))
            f.write(str(len(ws)) + ' ')
            f.flush()

            # update state
            x_sim.append(solution_ws.variables['x'][1] + e_t)
            f.write('%.3f'%np.linalg.norm(e_t) + ' ' + str(e_t) + '\n')
            f.flush()

        if rollout_success:
            nodes.append(nodes_i)
            nodes_ws.append(nodes_ws_i)
#             nodes_gurobi.append(nodes_gurobi_i)
            len_ws.append(len_ws_i)
            errors.append(errors_i)
            np.save('data/nodes_sd_%.3f'%e_sd, nodes)
            np.save('data/nodes_ws_sd_%.3f'%e_sd, nodes_ws)
            np.save('data/len_ws_sd_%.3f'%e_sd, len_ws)
            np.save('data/errors_sd_%.3f'%e_sd, errors)
        else:
            controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

        i += 1